* crossmatch all ur lists (original realsearch dipsearcplot, Fei's parsing, my reparsing of the vet pickles plots)                                                                     
  
* make list of: "probably planets", "probably EBs" "low SNR maybes"

In [1]:
import pandas as pd
import numpy as np

cl_path = '../results/real_search/merged_cand_lists/'

first = pd.read_csv(cl_path+'170410_human_labels.txt',
                    names=['kicid','human_label'], 
                    delimiter=',',dtype={'kicid':int,'human_label':str})
first = first[(first['human_label'] == 'dip') | (first['human_label'] == 'maybe_dip')]

second = pd.read_csv(cl_path+'170708_human_labels.txt',
                    names=['kicid','human_label'], 
                    delimiter=',',dtype={'kicid':int,'human_label':str})
second = second[(second['human_label'] == 'dip') | (second['human_label'] == 'maybe_dip')]

fei = pd.read_csv(cl_path+'fei_labels_170712.txt',
                    names=['kicid','comment'], 
                    delimiter=',',dtype={'kicid':int,'human_label':str})

vet = pd.read_csv(cl_path+'lgb_labels_vet_plot_170714.txt',
                    names=['kicid','comment'], 
                    delimiter=',',dtype={'kicid':int,'human_label':str})

In [2]:
len(first)

50

In [3]:
len(second)

110

make "probably planet" list

In [4]:
probplanet_list = []
probplanet_kicids = []

for cl in [first[first['human_label']=='dip'].iterrows(),
           second[second['human_label']=='dip'].iterrows(),
          fei[np.array([('planet' in str(f)) for f in fei['comment']])].iterrows(),
          vet[np.array([('planet' in str(f)) for f in vet['comment']])].iterrows()
          ]:
    for el in cl:
        probplanet_list.append(el[1]) # this is a stupid pandas iterator thing
        probplanet_kicids.append(int(el[1]['kicid']))

In [5]:
outdf = pd.DataFrame(probplanet_list)
outind = pd.DataFrame(probplanet_list).duplicated('kicid', keep='last')
outdf = outdf[~outind]
outdf.sort_values('kicid').head()

,comment,human_label,kicid
19,maybe planet,NaN,3245776
277,NaN,dip,5025261
24,planet(!),NaN,5302006
488,NaN,dip,5642620
327,NaN,dip,5881838


In [6]:
len(outdf)

35

make "interesting object" list

In [7]:
obj_list = []
obj_kicids = []

for co in [first[first['human_label']=='maybe_dip'].iterrows(),
           second[second['human_label']=='maybe_dip'].iterrows(),
          fei[np.array([('planet' not in str(f)) for f in fei['comment']])].iterrows(),
          vet[np.array([('planet' not in str(f)) for f in vet['comment']])].iterrows()
          ]:
    for el in co:
        obj_list.append(el[1]) # this is a stupid pandas iterator thing
        obj_kicids.append(int(el[1]['kicid']))

In [8]:
objdf = pd.DataFrame(obj_list)
objind = pd.DataFrame(obj_list).duplicated('kicid', keep='last')
objdf = objdf[~objind]
objdf.sort_values('kicid').head()

,comment,human_label,kicid
14,NaN,NaN,2449084
15,NaN,NaN,2557430
16,NaN,NaN,2695740
108,NaN,maybe_dip,2717141
17,NaN,NaN,2719436


In [9]:
len(objdf)

189

OK. A large number the candidates these lists will be BS (i.e. they'll disappear given improvements).

Let's go through them again in the vet plots and cull.

In [10]:
outdf.to_csv('../results/real_search/merged_cand_lists/170714_merged_dips_and_planets.csv', index=False)
objdf.to_csv('../results/real_search/merged_cand_lists/170714_merged_objs.csv', index=False)

In [11]:
alldf = pd.concat([outdf, objdf])
alldf.head()

,comment,human_label,kicid
32,NaN,dip,7515679
43,NaN,dip,9843451
273,NaN,dip,6791604
320,NaN,dip,5983351
0,NaN,dip,9790965


In [12]:
alldf = alldf.sort_values('kicid')

allind = alldf.duplicated('kicid', keep='last')

alldf[~allind].to_csv('../results/real_search/merged_cand_lists/170714_all_flagged.csv', index=False)

In [13]:
import os

In [16]:
for kicid in np.array(alldf['kicid']):
    
    if os.path.exists('/home/luke/Dropbox/proj/cbp/results/real_search/170714_all_flagged_symlinks/'+str(kicid)+'.png'):
        continue
    
    try:
        os.symlink('/home/luke/Dropbox/proj/cbp/results/vet_bls/'+str(kicid)+'.png',
                  '/home/luke/Dropbox/proj/cbp/results/real_search/170714_all_flagged_symlinks/'+str(kicid)+'.png')
    except:
        print('{:d} failed'.format(kicid))
    
    

2695740 failed
7733540 failed
7831363 failed
8822555 failed
11304987 failed


We might assume that these are all cases that didn't make the SNR > 3 cut in improved round 2 (170710) run of BLS.

Check by looking at original dipsearch plots individually.

* 502561 is noise
I will comment on any that are not noise:
* 2695740 had no dipsearchplot, originally...
* 7733540 definitely had some interesting dips in the first round. (Seen in relative flux mostly). In round 2 they were all low SNR, and the single >3 SNR recovery was a harmonic.
* 7831363 harmonic
* 8822555 not convinceable
* 11304987 all harmonics

So making the vet plot for 7733540, otherwise can parse from symlinks.